In [11]:
import glob
import os
import rdkit.Chem as Chem
import numpy as np
task_dirs = sorted(glob.glob("results/mini/task_*_repeat-5-monomers"))
local_monomers_all = []
global_monomers_all = []
R_list_all = []
t_list_all = []
psmiles_all = []
global_chain_all = []
for task_dir in task_dirs:
    local_monomer_sdf_files = sorted(glob.glob(os.path.join(task_dir, "local_monomer_*.sdf")))
    local_monomer_mols = [Chem.MolFromMolFile(path) for path in local_monomer_sdf_files]
    local_monomers_all.append(local_monomer_mols)
    global_monomer_sdf_files = sorted(glob.glob(os.path.join(task_dir, "global_monomer_*.sdf")))
    global_monomer_mols = [Chem.MolFromMolFile(path) for path in global_monomer_sdf_files]
    global_monomers_all.append(global_monomer_mols)
    global_chain = Chem.MolFromMolFile(os.path.join(task_dir,"homopoly_relaxed.sdf"))
    global_chain_all.append(global_chain)
    R_list = np.load(os.path.join(task_dir,"R_list.npy"))
    t_list = np.load(os.path.join(task_dir,"t_list.npy"))
    R_list_all.append(R_list)
    t_list_all.append(t_list)
    psmiles = ""
    with open(os.path.join(task_dir,"psmiles.txt"),"r") as f:
        psmiles = f.read()
    psmiles_all.append(psmiles)

In [16]:
from utils.chain_split_utils import remove_star_atoms
from utils.rigid_utils import to_local_coords,to_global_coords

# # 获取第0个分子的第n个原子的坐标（例如n=3）
# n = 3  # 你可以修改n为你想要的原子索引
# conf = local_monomer_mols[2].GetConformer(0)
# # 获取第0个分子的所有原子的坐标
# x0_list = []
# for i in range(local_monomer_mols[2].GetNumAtoms()):
#     pos = conf.GetAtomPosition(i)
#     x0_list.append([pos.x, pos.y, pos.z])
# x0_list = np.array(x0_list)
# x0_rotated = to_global_coords(x0_list,R_list[2],np.zeros(3))


# print(x0_rotated[s1],t_list[0])

In [17]:
# 这个函数无论传入什么分子，结果都一样，是因为它内部用的是外部变量conf（即local_monomer_mols[2]的构象），
# 而不是用传入的mol的构象。应该改为用mol.GetConformer()。
def get_mol_pos(mol:Chem.Mol):
    pos_list = []
    conf = mol.GetConformer(0)  # 正确获取当前mol的构象
    for i in range(mol.GetNumAtoms()):
        pos = conf.GetAtomPosition(i)
        pos_list.append([pos.x, pos.y, pos.z])
    return np.array(pos_list)

In [20]:
from copy import deepcopy
from utils.chain_split_utils import compose_monomers_from_local_monomers
from utils.conf_utils import set_coord_to_mol
from utils.rigid_utils import compute_rigid_frame_from_three_atoms
import copy


def calc_t_list(local_monomer_mols_list,R_list,psmiles):
    t_calc = [10,10,0]
    t_calc_list = [t_calc]
    key_point_list,processed_mol,processed_smi = remove_star_atoms(psmiles)
    n1,s1,n2,s2 = key_point_list
    for i in range(len(local_monomer_mols_list)):
        r = R_list[i]
        pos_local = get_mol_pos(local_monomer_mols_list[i])
        x1 = pos_local[n1]
        x2 = pos_local[s1]
        x3 = pos_local[s2]
        r_rec,t_rec = compute_rigid_frame_from_three_atoms(x1,x2,x3)
        local_coord = to_local_coords(pos_local,r_rec,t_rec)
        pos_rotated = to_global_coords(local_coord,r,t_calc)
        t_calc = pos_rotated[n2].tolist()
        t_calc_list.append(t_calc)
        # print(pos_rotated)
        # print(get_mol_pos(global_monomer_mols[i]))
        # print("--------------------------------")
    t_calc_list = np.array(t_calc_list)
    return t_calc_list,key_point_list
task_id = 0
t_calc_list,key_point_list = calc_t_list(local_monomers_all[task_id],R_list_all[task_id],psmiles_all[task_id])
global_chain_calc = compose_monomers_from_local_monomers(
    copy.copy(local_monomers_all[task_id]),R_list_all[task_id],t_calc_list,keypoints=key_point_list
)
from rdkit.Chem import AllChem

# # 使用rdkit自带的RMSD计算函数
rmsd = AllChem.GetBestRMS(copy.deepcopy(global_chain_all[task_id]), copy.deepcopy(global_chain_calc))
print(f"global_chain_calc 和 global_chain 的 RMSD: {rmsd:.4f}")

from utils.conf_utils import get_coord_from_mol

coords1 = np.array(get_coord_from_mol(global_chain))
coords2 = np.array(get_coord_from_mol(global_chain_calc))

global_chain_calc 和 global_chain 的 RMSD: 0.0001


In [23]:
import py3Dmol

def show_two_mols(mol1, mol2, colors=['blue', 'red']):
    mb1 = Chem.MolToMolBlock(mol1)
    mb2 = Chem.MolToMolBlock(mol2)
    view = py3Dmol.view(width=400, height=400)
    view.addModel(mb1, 'mol')
    view.addModel(mb2, 'mol')
    view.zoomTo()
    return view

view = show_two_mols(global_chain_all[task_id], global_chain_calc)
view.show()



3Dmol.js failed to load for some reason. Please check your browser console for error messages.